<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [17]</a>'.</span>

To do:
* importovat data -ok
* rozdelit na trenovacie, testovacie - ok
* data augmentation - ok
* importovat "cudzi" model bez poslednych vrstiev - ok
* pridat nove vrstvy - ok
* natrenovat model - aktualne chyba

* zlepsovat model (pozerat sa na uspesnost, co misklasifikuje najcastejsie...)
* mozno bude treba vycistit data a niektore obrazky vyhodit
* exportovat model - ok

In [ ]:
library(tidyverse)
library(keras)
library(tensorflow)
library(googledrive)
library(EBImage)
library(shiny)
library(readbitmap)
library("jpeg")

In [ ]:
img_size <- c(160, 160)
img_shape <- c(img_size, 3)
batch_size <- 32
epochs <- 5

Data

In [ ]:
train_dir <- "../input/nature/train/train"
validation_dir <- "../input/nature/test/test"

In [ ]:
flowers<-list.files("../input/nature/train/train/flower/",full.names = TRUE)
mushrooms<-list.files("../input/nature/train/train/mushroom/",full.names = TRUE)
trees<-list.files("../input/nature/train/train/tree/", full.names = TRUE)

Some images of flowers, mushrooms and trees

In [ ]:
par(mfrow=c(4, 4), mar=c(0, 0.2, 1, 0.2))
for (i in 1:16) {
  nr <- i * 10
  image<-readImage(flowers[nr])
    display(image)
}


In [ ]:
par(mfrow=c(4, 4), mar=c(0, 0.2, 1, 0.2))
for (i in 1:16) {
  nr <- i * 10
  image<-readImage(mushrooms[nr])
    display(image)
}


In [ ]:
par(mfrow=c(4, 4), mar=c(0, 0.2, 1, 0.2))
for (i in 1:16) {
  nr <- i * 10
  image<-readImage(trees[nr])
    display(image)
}


Data augmentation

In [ ]:
# tu je tiez moznost pohrat sa s parametrami otacania atd.
train_datagen <- image_data_generator(
    rotation_range = 15,
    rescale = 1/255,
    shear_range = 0.1,
    zoom_range = 0.2,
    horizontal_flip = TRUE,
    width_shift_range = 0.1,
    height_shift_range = 0.1
)
train_generator <- flow_images_from_directory(
    train_dir,
    generator = train_datagen,
    target_size = img_size,
    class_mode = 'categorical',
    batch_size = batch_size
)

validation_datagen <- image_data_generator(
    rescale=1/255)
validation_generator <- flow_images_from_directory(
  validation_dir,
  generator = validation_datagen,
  target_size = img_size,
  class_mode = 'categorical',
  batch_size = batch_size
)

In [ ]:
# number of training samples
train_samples <- train_generator$n
# number of validation samples
valid_samples <- validation_generator$n

Model - base model mobilenet V2

In [ ]:
base.model <- application_mobilenet_v2(
  input_shape = img_shape,
  include_top = FALSE,
  weights = 'imagenet')

In [ ]:
summary(base.model)

Model - transfer learning

In [ ]:
outputs <- base.model$output %>%
  layer_global_average_pooling_2d() %>%
  layer_dense(units = 126, activation = "relu") %>%
  layer_dropout(0.25) %>%
  layer_dense(units = 3, activation = "softmax")
# urcite sa treba pohrat s modelom a pridat nejake vrstvy,

model <- keras_model(inputs = base.model$input, outputs = outputs)

# freeze all layers of base model
for (layer in base.model$layers){layer$trainable <- FALSE}

# compile
model %>% compile(
  loss = "categorical_crossentropy",
  optimizer = optimizer_adam(),
  metrics = c("accuracy")
)


In [ ]:
history <- model %>%
  fit_generator(train_generator,
      steps_per_epoch = as.integer(train_samples / batch_size),
      epochs = epochs,
      validation_data = validation_generator,
      validation_steps = as.integer(valid_samples / batch_size),
      verbose = 2)


In [ ]:
summary(model)

In [ ]:
plot(history)

In [ ]:
history

****Save model

In [ ]:
save_model_tf(model, "nature-mod")

In [ ]:
## kdyby nekdo vedel
model %>% evaluate(flow_images_from_directory(
  validation_dir,
  target_size = img_size,
  class_mode = 'categorical',
  batch_size = batch_size)[1])

In [ ]:
mat<-apply(validation_generator[2][[2]],1,which.max)
mat

In [ ]:
# find misclassified pictures
predictions = model %>% predict(validation_generator[2])
pred <- to_categorical(apply(predictions, 1, which.max) - 1, 3)
#confusion matrix
table(pred, validation_generator[2][[2]])


In [ ]:
labeled_predict <- data.frame(cbind(as.matrix(mat), pred))
labeled_predict